<a href="https://colab.research.google.com/github/aya-se/advanced-machine-learning-2022/blob/main/aml2022_report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Machine Learning (2022) Final Report Assignment

Answer Questions 1 to 4 (either in Japanese or English). Submit a report in either PDF (.pdf) or JupyterNotebook (.ipynb) format.

## Question 1 (50 points)

Consider a convolutional neural network (CNN) that predicts a label $\hat{y} \in \{0, 1\}$ for a given sentence $\boldsymbol{X} \in \mathbb{R}^{d \times T}$. Here, a sentence is represented by a matrix $\boldsymbol{X} = (\boldsymbol{x}_1, \boldsymbol{x}_2, \dots, \boldsymbol{x}_T)$ consisting of a concatenation of $T$ word embeddings, $\boldsymbol{x}_1, \boldsymbol{x}_2, \dots, \boldsymbol{x}_T \in \mathbb{R}^d$, where $d$ is the size of word embeddings, and $T$ is the number of words in the sentence.

These equations define the whole architecture of the CNN.

\begin{align}
\hat{y} &= \begin{cases}
1 & (0.5 < p) \\
0 & (p \leq 0.5)
\end{cases} \\
p &= \sigma(\boldsymbol{v}^\top \boldsymbol{s}) \\
\boldsymbol{s} &= \max(\boldsymbol{c}_1, \dots, \boldsymbol{c}_{T-\delta+1}) \\
\boldsymbol{c}_t &= {\rm ReLU}(\boldsymbol{W} \boldsymbol{x}_{t:t+\delta-1} + \boldsymbol{b}) & (\forall t \in \{1, \dots, T-\delta+1\}) \\
\boldsymbol{x}_{t:t+\delta-1} &= \boldsymbol{x}_{t} \oplus \boldsymbol{x}_{t+1} \oplus \dots \oplus \boldsymbol{x}_{t+\delta-1}
\end{align}

Here:

+ $\boldsymbol{W} \in \mathbb{R}^{m \times \delta d}$, $\boldsymbol{b} \in \mathbb{R}^m, \boldsymbol{v} \in \mathbb{R}^m$ are the model parameters;
+ $m$ denotes the number of output channels of the CNN;
+ $\delta$ denotes the width (kernel size) of the convolution;
+ $\sigma(\cdot)$ denotes the standard sigmoid function;
+ $\max(\cdot)$ presents the max pooling operation;
+ ${\rm ReLU}(\cdot)$ denotes the ReLU activation function;
+ $\oplus$ presents a concatenation of vectors.

Setting the hyperparameters $d=3, m=2, \delta=2$, we initialize the model parameters as follows.

\begin{align}
\boldsymbol{W} &= \begin{pmatrix}
-3 & -2 & -1 & -1 & -2 & -3 \\
3 & 2 & 3 & 2 & 3 & 2
\end{pmatrix} \\
\boldsymbol{b} &= \begin{pmatrix}
-0.2 \\ 0.1
\end{pmatrix} \\
\boldsymbol{v} &= \begin{pmatrix}
-1 \\ 2
\end{pmatrix}
\end{align}

Suppose that we give a negative ($y=0$) training instance with the sentence ($T = 5$),

\begin{align}
\boldsymbol{X} &= \begin{pmatrix}
-0.3 & 0 & 0.1 & 0 & 0 \\
-0.2 & -0.1 & 0 & 0.1 & 0 \\
-0.1 & -0.2 & 0.1 & 0 & 0.1
\end{pmatrix} ,
\end{align}
to the CNN model, and answer the following questions.

**(1)** Find the value of the vector $\boldsymbol{x}_{3:4}$.

In [1]:
# 必要ライブラリのインポート
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# 各種変数の準備
W = np.array([[-3.0, -2.0, -1.0, -1.0, -2.0, -3.0], [3.0, 2.0, 3.0, 2.0, 3.0, 2.0]])
b = np.array([[-0.2], [0.1]])
v = np.array([[-1.0], [2.0]])
X = np.array([[-0.3, 0, 0.1, 0, 0], [-0.2, -0.1, 0, 0.1, 0], [-0.1, -0.2, 0.1, 0, 0.1]])
W, b, v, X

(array([[-3., -2., -1., -1., -2., -3.],
        [ 3.,  2.,  3.,  2.,  3.,  2.]]),
 array([[-0.2],
        [ 0.1]]),
 array([[-1.],
        [ 2.]]),
 array([[-0.3,  0. ,  0.1,  0. ,  0. ],
        [-0.2, -0.1,  0. ,  0.1,  0. ],
        [-0.1, -0.2,  0.1,  0. ,  0.1]]))

In [3]:
# x_{3:4}
np.concatenate([X[:, 2], X[:, 3]])

array([0.1, 0. , 0.1, 0. , 0.1, 0. ])

**(2)** Find the values of the hidden vectors $\boldsymbol{c}_1, \boldsymbol{c}_2, \boldsymbol{c}_3, \boldsymbol{c}_4$.

In [4]:
# コンテキストベクトルc_tの計算
def c(t) :
    val = (W @ np.concatenate([X[:, t], X[:, t+1]])).reshape(2, 1) + b
    return np.maximum(0, val)

c(0), c(1), c(2), c(3)

(array([[2.],
        [0.]]),
 array([[0.],
        [0.]]),
 array([[0.],
        [1.]]),
 array([[0. ],
        [0.5]]))

**(3)** Find the value of the vector $\boldsymbol{s}$.


In [5]:
s = np.max(np.hstack([c(0), c(1), c(2), c(3)]), axis=1)
s.reshape(2, 1)

array([[2.],
       [1.]])

**(4)** Find the value of $p$.

In [6]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

p = sigmoid(v.T @ s)[0]
p

0.5000000000000002

**(5)** Write the formula of the binary cross-entropy loss between the correct label $y$ and the probability estimate $p$.

In [7]:
# 注：p=0,1の時の処理は省略する
def BCE(y, p):
    return -(y * np.log(p) + (1-y) * np.log(1-p)) 

**(6)** Compute the loss value by using the formula of (5) for the training instance.

In [8]:
loss = BCE(0, p)
loss

0.6931471805599457

**(7)** Compute the gradient of the loss function with respect to $\boldsymbol{v}$ for the training instance.

負例の場合の勾配は、$\displaystyle\frac{\partial BCE(p)}{\partial v}=\frac{1}{1-p}\cdot p(1-p)\cdot s$と求まることから、以下のように計算できる。

In [9]:
grad_v = 1/(1-p) * p * (1-p) * s
grad_v.reshape(2, 1)

array([[1. ],
       [0.5]])

**(8)** Compute the gradients of the loss function with respect to $\boldsymbol{W}$ for the training instance.

(7)と同様にして誤差逆伝播法で、勾配を計算

In [10]:
grad_W = 1/(1-p) * p * (1-p) * v * np.array([np.concatenate([X[:, 0], X[:, 0+1]]), np.concatenate([X[:, 2], X[:, 2+1]])])
grad_W

array([[ 0.15,  0.1 ,  0.05, -0.  ,  0.05,  0.1 ],
       [ 0.1 ,  0.  ,  0.1 ,  0.  ,  0.1 ,  0.  ]])

## Question 2 (20 points)

Give names of two datasets that can be used to evaluate the quality of word embeddings, and explain the datasets with the following perspectives.

+ Brief explanation of the task for the evaluation.
+ Statistics of the dataset (e.g., the number of instances)
+ Measure(s) for evaluating the quality

### Natural Questions
- いわゆる、**オープンドメインの質問応答** (Open-Domain Question Answering) と呼ばれるタスクの代表的なデータセットの1つである。与えられた質問文に対する回答を、WikipediaなどのWeb文書上から探し出して回答する。一般に、Retriever(検索器)モデルで回答の証拠となりそうな文書を抽出し、それをもとにGenerator(生成器)やReader(スパン抽出器)で最終的な回答を生成したり、文書中から抽出したりする。これらのあらゆる過程で単語埋め込みの品質は重要となる。
- 学習データ：307,373個 / 検証データ：7,830 examples / テストデータ：7,842個
- Natural Questionsを含む、一般的なQAは回答が短答式であるため、参照回答と予測回答との**完全一致**(Exact Match・EM)をベースに評価することが基本となっている。

### QMSum
- 比較的新しい**要約**データセットの1つであり、議事録形式であること、さらにクエリ指向型であることが特徴である。一般的な要約タスクは、ある文書全体の要約を作成することが目標であるが、クエリ指向型要約では、特定のクエリ(質問・お題)に沿った要約を作成することが求められる (例：「Aさんは今期の売上業績についてどのような意見を述べていますか？」など)。また、議事録は対話データであり、一般的な文書データよりも合計トークン数が多い、対話特有の構造や談話内容の理解が難しい、などの性質から、QMSumは多くの要約データセットの中でもタスクとしての難易度が高い部類に相当すると考えられる。
- 学習データ：1,257個 / 検証データ：272個 / テストデータ：279個 (※クエリとソース文のペアでカウント)
- データ数自体はかなり少ないが、1つ1つの会議の平均トークン長は9,069.8であり、非常に長い
- QMSumを含む、一般的な要約タスクは、参照回答と予測回答との**ROUGE**スコアで評価することが基本となっている。これは両者間の語彙重複を評価する指標であり、**ROUGE-1・ROUGE-2・ROUGE-L・ROUGE-Lsum**などの種類がある。ただし、ROUGEスコアのみだと、実際の生成文の流暢性・事実への忠実性・情報量などの観点での評価ができないため、自動評価に加えて、**人間による評価**を合わせて実施することも多い。


## Question 3 (20 points)

Explain two reasons why Transformers are superior to Recurrent Neural Network
(RNN) in sequence-to-sequence tasks such as Machine Translation.

- RNNでは時系列データの直前までの情報を1つの隠れベクトルとして埋め込むことになるが、これに埋め込むことのできる情報量は限界がある。また、モデルの構造上、遠いトークン間の情報はどうしても伝達に難がある。一方でTransformerの場合は、基本的に全てのトークン対についてAttentionメカニズムで情報を伝達できるため、遠いトークン間を含めて、より豊富な情報量を表現することが可能である。自然言語処理では、長距離の文脈を考慮しなければならないようなタスクも多いため、そのような場合には特に、TransformerはRNNに対する高い優位性を持つと考えられる。

- RNNはある時刻上での隠れベクトルを逐次的に計算していく必要があるが、GPUの並列化によって計算時間を短縮することが重要となる深層学習において、この性質は非常にネックである。一方でTransformerは少なくとも学習時であれば、全てのAttentionの計算は独立に行えるため、計算の並列化が行いやすい。従って計算の並列化の面でも、TransformerはRNNに対する優位性があるといえる。(ただし、TransformerのAttentionも入力長に対して$O(N^2)$の計算量・メモリが必要となるため、非常に長い入力を扱う際には工夫が必要となる。)

## Question 4 (10 points)

Implement the code for using a pre-trained **language** model. Show the code and its output as well as the following information:

+ The detail of the pre-trained language model, for example,
    + https://huggingface.co/EleutherAI/gpt-j-6B
    + https://huggingface.co/rinna/japanese-gpt-1b
    + https://huggingface.co/facebook/blenderbot-400M-distill
+ The task addressed by the model (e.g., "text generation", "summarization", "chatbot")


HuggingFaceのBARTに岡崎研究室の紹介文を流し込んだだけ。

In [13]:
# HuggingFaceから要約用にファインチューニングされたBARTの生成モデルを使用
# https://huggingface.co/philschmid/bart-large-cnn-samsum
from transformers import pipeline
summarizer = pipeline("summarization", model="philschmid/bart-large-cnn-samsum")

ARTICLE_TO_SUMMARIZE = "We hope the students joined the laboratory to acquire skills through research activities and make a confident start as a member of society in the future. \
                        Once you start working in the community, you are not only required to solve the problem of your clients or others, \
                        but also to spot or suggest a potential issue which even the client doesn’t notice. When you write a bachelor/master thesis, \
                        you also need to explore the unsolved problems by yourself, address them by all possible means to search for a solution, and give your findings back to the society. \
                        Through your research, you can acquire abilities to establish your specialty as if building your castle, to complete theories/works, and to appeal to others, \
                        which would be useful anywhere in the society. There is always someone who has been conducting a related study in the world. \
                        Whenever you advance your research, you will face challenges from the world. For this reason, even when you write a bachelor/master thesis, \
                        you should set the research topic bearing in mind the need to present the findings of your research in international conferences or journals. \
                        The laboratory has diverse people with various hobbies, skills, and thoughts; we respect the individuality of each member, learn from others, and have fun together. \
                        We hope that your experiences in the laboratory will be invaluable for the rest of your lives."

summarizer(ARTICLE_TO_SUMMARIZE)

vocab_file vocab.json
merges_file merges.txt
tokenizer_file tokenizer.json
added_tokens_file added_tokens.json
special_tokens_map_file special_tokens_map.json
tokenizer_config_file tokenizer_config.json


[{'summary_text': 'The students joined the laboratory to acquire skills through research activities and make a confident start as a member of society in the future. The laboratory has diverse people with various hobbies, skills, and thoughts. They respect the individuality of each member, learn from others, and have fun together.'}]

### (参考)日本語訳
#### 原文
研究室には、研究活動を通じてスキルを身につけ、将来、社会人として自信を持ってスタートできるような人材を求めて入室してもらいました。社会で仕事をするようになると、クライアントなどの問題を解決するだけでなく、クライアントも気づいていない潜在的な問題を発見し、提案することが求められます。また、学士・修士論文では、未解決の問題を自ら掘り起こし、あらゆる手段を使って解決策を模索し、その成果を社会に還元することが求められます。研究を通して、自分の城を築くように専門性を確立し、理論や作品を完成させ、社会にアピールする能力を身につけることができ、それは社会のどこでも通用するものです。

世の中には、必ず関連する研究をしている人がいます。研究を進めると、必ず世の中の課題に直面する。ですから、学士・修士論文を書く場合でも、研究成果を国際会議や学術誌で発表することを念頭に置いて、研究テーマを設定する必要があります。

研究室には、様々な趣味、技術、考えを持った人が集まっており、それぞれの個性を尊重し、他者から学び、共に楽しむことができます。研究室での経験が、皆さんの生涯に渡ってかけがえのないものになることを願っています。

#### 要約結果
研究活動を通じて技術を身につけ、将来、社会人として自信を持ってスタートを切るために、研究室に入りました。研究室には、さまざまな趣味や特技、考えを持った人が集まっています。それぞれの個性を尊重し、他者から学び、共に楽しみます。

---

ちょっとニュアンスが怪しいが、まぁだいだい妥当な要約にはなっている。
